In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import svr_IAM
%load_ext autoreload
%autoreload 2

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-pastel')

### 1. Load PAGE, FUND, DICE

In [ ]:
fund_data= pd.read_csv("FUND/FUND_socio_scc_complete.csv", index_col=0)
fund_data

In [ ]:
fund_data.columns

'SCC_default', 
'cs2.81_prtp1.5', 'cs3.81_prtp75_prtp1.5', 'cs11.12_prtp1.5',
'cs2.81_prtp1.03', 'cs3.81_prtp175_prtp1.03', 'cs11.12_prtp1.03',
'cs2.81_prtp3.0', 'cs3.81_prtp75_prtp3.0', 'cs11.12_prtp3.0',

In [ ]:
page_data= pd.read_csv("PAGE/PAGE_socio_scc_complete.csv", index_col=0)
page_data

In [ ]:
dice_data= pd.read_csv("DICE/DICE_socio_scc_complete.csv", index_col=0)
dice_data

### Pre-processing each IAM data

#### 1. FUND

In [ ]:
fund_data_use= fund_data.loc[(fund_data["Year"] >= 2020) & (fund_data["Year"] <= 2100)].reset_index(drop=True)
fund_data_use

In [ ]:
fund_data_current= fund_data_use.copy()
fund_data_current.columns

In [ ]:
fund_data_current.iloc[:, -13:]= fund_data_current.iloc[:, -13:] * 1.98
fund_data_current

#### 2. PAGE

In [ ]:
page_data_current= page_data.copy()

page_data_current.iloc[:, -13:]= page_data_current.iloc[:, -13:] * 1.27
page_data_current


In [ ]:
page_data_current["popu"]= page_data_current["popu"]/1000

In [ ]:
page_data_current

In [ ]:
fund_data_current.columns

In [ ]:
page_data_current.columns = ['Year', 'popu', 'GDP_percap', 'emiss_total', 'SCC', 'PAGE_1', 'PAGE_2', 'PAGE_3', 'PAGE_4', 
                                                                                 'PAGE_5', 'PAGE_6', 'PAGE_7', 'PAGE_8', 'PAGE_9',
                                                                                 'PAGE_10', 'PAGE_11', 'PAGE_12']

In [ ]:
page_data_current

#### 3. DICE

In [ ]:
dice_data_current= dice_data.copy()

dice_data_current.iloc[:, -13:]= dice_data_current.iloc[:, -13:] * 1.5
dice_data_current


In [ ]:
dice_data_current.rename(columns={"emiss_tot": "emiss_total"}, inplace=True)

In [ ]:
dice_data_current["popu"]= dice_data_current["popu"]/1000

In [ ]:
dice_data_current

### 2. Make X and y dataset

In [ ]:
PAGE_X_df= page_data_current.iloc[:, :4]
PAGE_y_df= page_data_current.iloc[:, -13:]

FUND_X_df= fund_data_current.iloc[:, :4]
FUND_y_df= fund_data_current.iloc[:, -13:]

DICE_X_df= dice_data_current.iloc[:, :4]
DICE_y_df= dice_data_current.iloc[:, -13:]


In [ ]:
PAGE_X_df.columns, PAGE_y_df.columns

In [ ]:
FUND_X_df.columns, FUND_y_df.columns

In [ ]:
DICE_X_df.columns, DICE_y_df.columns

In [ ]:
PAGE_y_df.columns= ['SCC', 'SCC_1', 'SCC_2', 'SCC_3', 'SCC_4', 'SCC_5', 'SCC_6', 
                            'SCC_7', 'SCC_8', 'SCC_9', 'SCC_10', 'SCC_11', 'SCC_12']

FUND_y_df.columns= ['SCC', 'SCC_1', 'SCC_2', 'SCC_3', 'SCC_4', 'SCC_5', 'SCC_6', 
                            'SCC_7', 'SCC_8', 'SCC_9', 'SCC_10', 'SCC_11', 'SCC_12']

DICE_y_df.columns= ['SCC', 'SCC_1', 'SCC_2', 'SCC_3', 'SCC_4', 'SCC_5', 'SCC_6', 
                            'SCC_7', 'SCC_8', 'SCC_9', 'SCC_10', 'SCC_11', 'SCC_12']

In [ ]:
# PAGE_y_df, 
FUND_y_df

In [ ]:
FUND_y_df.columns

In [ ]:
X_df= pd.concat([FUND_X_df, PAGE_X_df, DICE_X_df]).reset_index(drop=True)
X_df

In [ ]:
y_df= pd.concat([FUND_y_df, PAGE_y_df, DICE_y_df]).reset_index(drop=True)
y_df

In [ ]:
y_df_prtp15= y_df[['SCC_1', 'SCC_2', 'SCC_3', 'SCC_4']]
y_df_prtp1= y_df[['SCC_5', 'SCC_6', 'SCC_7', 'SCC_8']]
y_df_prtp3= y_df[['SCC_9', 'SCC_10', 'SCC_11', 'SCC_12']]

### 3. Scaling

In [ ]:
# prtp_name= "prtp_15"
# y_df_use= y_df_prtp15.copy()

# prtp_name= "prtp_1"
# y_df_use= y_df_prtp1.copy()

prtp_name= "prtp_3"
y_df_use= y_df_prtp3.copy()


In [ ]:
len(y_df_prtp1), len(y_df_prtp15)

In [ ]:
import os
''' making new directory folder '''
def making_dir(dir_name):
    # https://thispointer.com/how-to-create-a-directory-in-python/
    # Create target Directory if don't exist
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        print("Directory " , dir_name ,  " Created ")
    else:    
        print("Directory " , dir_name ,  " already exists")
    
    return

In [ ]:
svr_prtp_folder= "SVR_01_1030_" + prtp_name

making_dir(dir_name= svr_prtp_folder)

In [ ]:
X_df.iloc[:, 1:]

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def scaling_transform(choose_scaler, for_scale, inverse=None, for_inverse=None):
    
    if choose_scaler == "minmax":
        scaler = MinMaxScaler()
        data_scaled = scaler.fit_transform(for_scale)
    else:
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(for_scale)


    if inverse == 'yes':
        data_inverse = scaler.inverse_transform(for_inverse)
        return data_inverse
    else:
        return data_scaled

In [ ]:
scaler= 'minmax'

X_scld= scaling_transform(choose_scaler= scaler, for_scale= X_df.iloc[:, 1:])
y_scld= scaling_transform(choose_scaler= scaler, for_scale= y_df_use)

In [ ]:
X_scld, X_scld.shape

In [ ]:
y_scld, y_scld.shape

In [ ]:
### add year

X_scld_add= np.hstack([X_df.Year.values.reshape(-1,1), X_scld])
X_scld_add

In [ ]:
all_index= X_df.index.values
all_index

In [ ]:
import random
test_index= random.choices(all_index, k= 10)
test_index, len(test_index)

In [ ]:
training_index= np.delete(all_index, test_index)
training_index, len(training_index)

In [ ]:
X_training_scld= X_scld_add[training_index]
X_training_scld

y_training_scld= y_scld[training_index]
y_training_scld

In [ ]:
X_training_scld_02= np.hstack((training_index.reshape(-1,1), X_training_scld))
y_training_scld_02= np.hstack((training_index.reshape(-1,1), y_training_scld))


In [ ]:
X_test_scld= X_scld_add[test_index]
X_test_scld

y_test_scld= y_scld[test_index]
y_test_scld

In [ ]:
X_test_scld_02= np.hstack((np.asarray(test_index).reshape(-1,1), X_test_scld))
y_test_scld_02= np.hstack((np.asarray(test_index).reshape(-1,1), y_test_scld))


In [ ]:
print(X_training_scld_02.shape, y_training_scld_02.shape, 
      X_test_scld_02.shape, y_test_scld_02.shape)

### 4. Split data

In [ ]:
from sklearn.model_selection import train_test_split

X_train_scld, X_val_scld, y_train_scld, y_val_scld= train_test_split(X_training_scld_02, y_training_scld_02)

In [ ]:
print(X_train_scld.shape, X_val_scld.shape, y_train_scld.shape, y_val_scld.shape)

In [ ]:
id_train= X_train_scld[:, 0]
id_val= X_val_scld[:, 0]
id_test= X_test_scld_02[:,0]

In [ ]:
svr_prtp_folder

In [ ]:
np.savetxt(svr_prtp_folder + "/" + "id_train.txt", id_train)
np.savetxt(svr_prtp_folder + "/" + "id_val.txt", id_val)
np.savetxt(svr_prtp_folder + "/" + "id_test.txt", id_test)

In [ ]:
year_train= X_train_scld[:, 1]
year_val= X_val_scld[:, 1]
year_test= X_test_scld_02[:,1]

In [ ]:
np.savetxt(svr_prtp_folder + "/" + "X_train.txt", X_train_scld)
np.savetxt(svr_prtp_folder + "/" + "y_train.txt", y_train_scld)

np.savetxt(svr_prtp_folder + "/" + "X_val.txt", X_val_scld)
np.savetxt(svr_prtp_folder + "/" + "y_val.txt", y_val_scld)

np.savetxt(svr_prtp_folder + "/" + "X_test.txt", X_test_scld)
np.savetxt(svr_prtp_folder + "/" + "y_test.txt", y_test_scld)

In [ ]:
y_df

In [ ]:
X_df

In [ ]:
y_df

In [ ]:
y_df.columns

In [ ]:
y_df_year= pd.concat([y_df, X_df['Year']], axis=1)
y_df_year.columns

In [ ]:
y_df_year

In [ ]:
X_train= X_df.iloc[id_train]
y_train= y_df_use.iloc[id_train]
X_train

In [ ]:
X_val= X_df.iloc[id_val]
y_val= y_df_use.iloc[id_val]
X_val

In [ ]:
X_test= X_df.iloc[id_test]
y_test= y_df_use.iloc[id_test]
X_test

In [ ]:
X_train.to_csv(svr_prtp_folder + "/" + "X_train_df_actual.csv")
y_train.to_csv(svr_prtp_folder + "/" + "y_train_df_actual.csv")

X_val.to_csv(svr_prtp_folder + "/" + "X_val_df_actual.csv")
y_val.to_csv(svr_prtp_folder + "/" + "y_val_df_actual.csv")

X_test.to_csv(svr_prtp_folder + "/" + "X_test_df_actual.csv")
y_test.to_csv(svr_prtp_folder + "/" + "y_test_df_actual.csv")

### 5. SVR

In [ ]:
def hyperparam_list(C_max=50, C_ival=1, epsgam_max=10, epsgam_ival=1):
    # start_list = np.arange(0, 10, 0.5)
    # b = [0.1**x for x in start_list]

    # print(start_list)
    # print(b)

    C_list = np.arange(1, C_max, C_ival)
    print(C_list)
    print(C_list.shape)

    start_list = np.arange(0, epsgam_max, epsgam_ival)
    eps_list = np.array([0.1**x for x in start_list])
    print(eps_list)
    print(eps_list.shape)

    # gam_list = np.arange(0.1, 10, 0.1)
    gam_list = np.array([0.1**x for x in start_list])
    print(gam_list)
    print(gam_list.shape)
    return C_list, eps_list, gam_list

In [ ]:
""" Create list of hyperparameter C, epsilon, gamma """

C_list, eps_list, gam_list= hyperparam_list()

In [ ]:
add_name= "_shuffle_1030_3iams_" + prtp_name
add_name

In [ ]:
from datetime import datetime
start_time = datetime.now()

np.random.seed(42)



tuning_df= svr_IAM.fine_tune_SVR(C_list= C_list,
                                 eps_list= eps_list,
                                 gam_list= gam_list,
                                 X_train= X_train_scld[:, 2:],
                                 y_train= y_train_scld[:, 1:],
                                 X_val= X_val_scld[:, 2:],
                                 y_val= y_val_scld[:, 1:],
                                 SVR_folder= svr_prtp_folder,
                                 add_name= add_name)


end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
rmse_param= svr_IAM.best_param_use(results_df= tuning_df,
                                   train_size= len(X_train_scld),
                                   val_size= len(X_val_scld),
                                   scaled_info= scaler + "_scale",
                                   SVR_folder= svr_prtp_folder,
                                   add_name= add_name,
                                   use_param='RMSE_val',
                                   )

In [ ]:
add_name

In [ ]:
y_train_pred, y_val_pred= svr_IAM.train_SVR(all_best= rmse_param,
                                            X_train= X_train_scld[:, 2:],
                                            y_train= y_train_scld[:, 1:],
                                            X_val= X_val_scld[:, 2:],
                                            SVR_folder= svr_prtp_folder,
                                            add_name= add_name
                                            )

In [ ]:
np.savetxt(svr_prtp_folder + "/" + "y_train_pred_scld.txt", y_train_pred)
np.savetxt(svr_prtp_folder + "/" + "y_val_pred_scld.txt", y_val_pred)

In [ ]:
""" RMSE """

train_rmse = svr_IAM.RMSE_calc(actual= y_train_scld[:, 1:], 
                               prediction= y_train_pred)
val_rmse = svr_IAM.RMSE_calc(actual= y_val_scld[:, 1:], 
                             prediction= y_val_pred)

train_r2 = svr_IAM.r2_calc(actual= y_train_scld[:, 1:], prediction= y_train_pred)
val_r2 = svr_IAM.r2_calc(actual= y_val_scld[:, 1:], prediction= y_val_pred)


print(train_rmse, val_rmse)
print(train_r2, val_r2)

In [ ]:
cl_list = ["gold", "orange", "saddlebrown",
            "skyblue", "royalblue", "midnightblue",
            "plum", "darkmagenta",  "darkviolet"
            ]

In [ ]:
scc_col_name_ls= y_df.columns
scc_col_name_ls

In [ ]:
def plot_pred_actual(year, y_actual, y_pred, title,
                     y_label= " ", ymin=None, ymax=None, legend_loc= "upper left",
                     save=False, savename=None, ncol=1, s=10,
                     W= 10, H=7, fs=25, pad=15):

    plt.figure(figsize=(W,H))

    plt.plot(year, y_actual, marker="o", markersize=s, linestyle="None", label= "Actual")
    plt.plot(year, y_pred, marker="^", markersize=s-2, linestyle="None", label= "Prediction")

    plt.title(title, fontsize=fs, pad=pad)

    plt.xlabel("Year", fontsize= fs-5)
    plt.ylabel(y_label, fontsize= fs-5)
    plt.ylim(ymin, ymax)
    plt.tick_params("both", labelsize=fs-5)

    plt.legend(fontsize= fs-5, ncol=ncol, loc=legend_loc)
    plt.tight_layout()

    if save == True:
        plt.savefig(savename + ".png", bbox_inches="tight")

    plt.show()

In [ ]:
len(fund_data)

In [ ]:
len(X_df)

In [ ]:
y_train_scld[:, 0]

In [ ]:
fund_id_train= np.where(y_train_scld[:, 0] < len(fund_data_use))
fund_id_val= np.where(y_val_scld[:, 0] < len(fund_data_use))

In [ ]:
fund_id_train

In [ ]:
y_train_scld[fund_id_train].shape

In [ ]:
n= 2

plot_pred_actual(year= X_train_scld[fund_id_train][:, 1], 
                 y_actual= y_train_scld[fund_id_train][:, n],
                 y_pred= y_train_pred[fund_id_train][:, n-1],
                 title= "Prediction for train ",
                 save=True,
                 savename= svr_prtp_folder + "/" + "pred_train_scld_" + scc_col_name_ls[n]
                 )

In [ ]:
len(page_data)

In [ ]:
len(fund_data_use)

In [ ]:
len(page_data_current)

In [ ]:
page_id_train= np.where((y_train_scld[:, 0] >= len(fund_data_use)) & (y_train_scld[:, 0] < len(fund_data_use)*2))
page_id_val= np.where((y_val_scld[:, 0] >= len(fund_data_use)) & (y_val_scld[:, 0] < len(fund_data_use)*2))

In [ ]:
# page_id_train= np.where(y_train_scld[:, 0] >= len(fund_data_use))
# page_id_val= np.where(y_val_scld[:, 0] >= len(fund_data_use))

In [ ]:
n= 1

plot_pred_actual(year= X_train_scld[page_id_train][:, 1], 
                 y_actual= y_train_scld[page_id_train][:, n],
                 y_pred= y_train_pred[page_id_train][:, n-1],
                 title= "Prediction for train ",
                 save=True,
                 savename= svr_prtp_folder + "/" + "pred_train_scld_" + scc_col_name_ls[n]
                 )

In [ ]:
dice_id_train= np.where((y_train_scld[:, 0] >= len(fund_data_use)*2))
dice_id_val= np.where((y_val_scld[:, 0] >= len(fund_data_use)*2))

In [ ]:
n= 1

plot_pred_actual(year= X_train_scld[dice_id_train][:, 1], 
                 y_actual= y_train_scld[dice_id_train][:, n],
                 y_pred= y_train_pred[dice_id_train][:, n-1],
                 title= "Prediction for train ",
                 save=True,
                 savename= svr_prtp_folder + "/" + "pred_train_scld_" + scc_col_name_ls[n]
                 )

In [ ]:
y_train_inv = svr_IAM.scaling_transform(choose_scaler= scaler,
                                        for_scale= y_df_use,
                                        inverse='yes',
                                        for_inverse= y_train_scld[:, 1:])


y_val_inv = svr_IAM.scaling_transform(choose_scaler= scaler,
                                      for_scale= y_df_use,
                                      inverse='yes',
                                      for_inverse= y_val_scld[:, 1:])


y_train_pred_inv = svr_IAM.scaling_transform(choose_scaler= scaler,
                                            for_scale= y_df_use,
                                            inverse='yes',
                                            for_inverse= y_train_pred)


y_val_pred_inv = svr_IAM.scaling_transform(choose_scaler= scaler,
                                           for_scale= y_df_use,
                                           inverse='yes',
                                           for_inverse= y_val_pred)


In [ ]:
np.savetxt(svr_prtp_folder + "/" + "y_train_pred_inv.txt", y_train_pred_inv)
np.savetxt(svr_prtp_folder + "/" + "y_val_pred_inv.txt", y_val_pred_inv)

In [ ]:
n= 1

plot_pred_actual(year= X_train_scld[fund_id_train][:, 1], 
                 y_actual= y_train.iloc[fund_id_train].iloc[:, n-1],
                 y_pred= y_train_pred_inv[fund_id_train][:, n-1],
                 title= "Prediction for train ",
                 save=True,
                 savename= svr_prtp_folder + "/" + "pred_train_scld_" + scc_col_name_ls[n]
                 )

### 6. Load model and prediction test data

In [ ]:
import joblib

In [ ]:
""" LOAD MODEL """

model= joblib.load(svr_prtp_folder + "/" + add_name + "_SVR_model.pkl")

In [ ]:
y_test_pred= model.predict(X_test_scld[:, 1:])

In [ ]:
y_test_inv = svr_IAM.scaling_transform(choose_scaler= scaler,
                                        for_scale= y_df_use,
                                        inverse='yes',
                                        for_inverse= y_test_scld)
                                        
y_test_pred_inv = svr_IAM.scaling_transform(choose_scaler= scaler,
                                            for_scale= y_df_use,
                                            inverse='yes',
                                            for_inverse= y_test_pred)

In [ ]:
np.savetxt(svr_prtp_folder + "/" + "y_test_pred_scld.txt", y_test_pred)
np.savetxt(svr_prtp_folder + "/" + "y_test_pred_inv.txt", y_test_pred_inv)

In [ ]:
""" RMSE """

test_rmse = svr_IAM.RMSE_calc(actual= y_test_scld, prediction= y_test_pred)
print(test_rmse)



""" R2 """

test_r2 = svr_IAM.r2_calc(actual= y_test_scld, prediction= y_test_pred)
print(test_r2)

# """ MAPE """

# test_mape = svr_IAM.MAPE_calc(actual= y_test_scld, prediction= y_test_pred)
# print(test_mape)

In [ ]:
eval_metric_df= pd.DataFrame({"Data": ["train", 'val', 'test'],
                              "RMSE": [train_rmse, val_rmse, test_rmse],
                              "R2": [train_r2, val_r2, test_r2]})

eval_metric_df

In [ ]:
# test_rmse_r2_df= pd.DataFrame({'RMSE': [test_rmse], 'R2': [test_r2]})
# test_rmse_r2_df

In [ ]:
# test_rmse_r2_df.to_csv(svr_prtp_folder + "/" + "test_rmse_r2.csv")

In [ ]:
eval_metric_df.to_csv(svr_prtp_folder + "/" + "eval_metric_train_val_test.csv")

In [ ]:
n=0

plot_pred_actual(year= year_test, 
                 y_actual= y_test.iloc[:, n],
                 y_pred= y_test_pred_inv[:, n],
                 y_label= "Current " + "$ USD/t CO_2$",
                 title= "Prediction for test " + scc_col_name_ls[n],
                 save=True,
                 savename= svr_prtp_folder + "/" + "pred_test_inv_" + scc_col_name_ls[n])

In [ ]:
pred_inv_combine= np.vstack([y_train_pred_inv, y_val_pred_inv, y_test_pred_inv])
pred_inv_combine.shape

In [ ]:
index_all= np.vstack([id_train.reshape(-1,1), id_val.reshape(-1,1), id_test.reshape(-1,1)])

In [ ]:
year_all= np.hstack([year_train, year_val, year_test])
year_all

In [ ]:
pred_inv_combine_df= pd.concat([pd.DataFrame(data= index_all, columns= ["ID"]),
                                pd.DataFrame(data= year_all, columns= ["Year"]),
                                pd.DataFrame(data= pred_inv_combine, columns= y_df_use.columns)], axis=1).sort_values(by=["ID"]).reset_index(drop=True)


In [ ]:
pred_inv_combine_df

In [ ]:
fund_data_pred_inv= pred_inv_combine_df.iloc[:len(fund_data_use)]
fund_data_pred_inv

### Plotting

In [ ]:
col_use= 'SCC_12'
plt.plot(fund_data_pred_inv['Year'], fund_data_pred_inv[col_use], 
         marker="o", markersize=10, linestyle="None", label= "Prediction")

plt.plot(fund_data_pred_inv['Year'], FUND_y_df[col_use], 
         marker="^" , markersize=10, linestyle="None", label= "Actual")

plt.legend()

In [ ]:
fund_data_pred_inv.to_csv(svr_prtp_folder + "/" + "fund_data_pred_inv_all.csv")

In [ ]:
page_data_pred_inv= pred_inv_combine_df.iloc[len(fund_data_use):len(fund_data_use)*2]
page_data_pred_inv, page_data_pred_inv.shape

In [ ]:
col_use= 'SCC_12'
plt.plot(page_data_pred_inv['Year'], page_data_pred_inv[col_use], 
         marker="o", markersize=10, linestyle="None", label= "Prediction")

plt.plot(page_data_pred_inv['Year'], PAGE_y_df[col_use], 
         marker="^" , markersize=10, linestyle="None", label= "Actual")

plt.legend()

In [ ]:
page_data_pred_inv

In [ ]:
page_data_pred_inv.to_csv(svr_prtp_folder + "/" + "page_data_pred_inv_all.csv")

In [ ]:
dice_data_pred_inv= pred_inv_combine_df.iloc[len(fund_data_use)*2:len(fund_data_use)*3]
dice_data_pred_inv, dice_data_pred_inv.shape

In [ ]:
col_use= 'SCC_12'
plt.plot(dice_data_pred_inv['Year'], dice_data_pred_inv[col_use], 
         marker="o", markersize=10, linestyle="None", label= "Prediction")

plt.plot(dice_data_pred_inv['Year'], DICE_y_df[col_use], 
         marker="^" , markersize=10, linestyle="None", label= "Actual")

plt.legend()

In [ ]:
def plot_compare_final(col_use, pred_inv_data, actual_y_data, 
                       title, y_label= " ", ymin=None, ymax=None, 
                       legend_loc= "upper left",
                       save=False, savename=None, ncol=1, s=10,
                       L=7, H=5, fs=20, pad=7):
    
    plt.figure(figsize=(L,H))
    plt.plot(pred_inv_data['Year'], actual_y_data[col_use], 
             marker="^" , markersize=10, linestyle="None", label= "Actual", color= "lightgreen")
    plt.plot(pred_inv_data['Year'], pred_inv_data[col_use], 
             marker="o", markersize=10, linestyle="None", label= "Prediction", color= "lightblue")


    plt.title(title, fontsize=fs, pad=pad)

    plt.xlabel("Year", fontsize= fs-5)
    plt.ylabel(y_label, fontsize= fs-5)
    plt.ylim(ymin, ymax)
    plt.tick_params("both", labelsize=fs-5)

    plt.legend(fontsize= fs-5, ncol=ncol, loc=legend_loc)
    plt.tight_layout()

    if save == True:
        plt.savefig(savename + ".png", bbox_inches="tight", dpi=600)

    plt.show()


In [ ]:
def plot_compare_final_2(col_use_list, pred_inv_data, actual_y_data, 
                       title, marker_list, 
                       y_label= " ", ymin=None, ymax=None, 
                       legend_loc= "upper left", bbox_loc= (1.1,1),
                       save=False, savename=None, ncol=1,
                       L=7, H=5, fs=20, pad=7):
    
    plt.figure(figsize=(L,H))

    for i in range(len(col_use_list)):
        plt.plot(pred_inv_data['Year'], actual_y_data[col_use_list[i]], 
                 marker=marker_list[i] , markersize=10, color= "lightgreen",
                 linestyle="None", label= "Actual:" + col_use_list[i])
                
        plt.plot(pred_inv_data['Year'], pred_inv_data[col_use_list[i]], 
                 marker=marker_list[i], markersize=10, color= "lightblue",
                 linestyle="None", label= "Prediction: " + col_use_list[i])

        


    plt.title(title, fontsize=fs, pad=pad)

    plt.xlabel("Year", fontsize= fs-5)
    plt.ylabel(y_label, fontsize= fs-5)
    plt.ylim(ymin, ymax)
    plt.tick_params("both", labelsize=fs-5)

    plt.legend(fontsize= fs-7, ncol=ncol, loc=legend_loc, bbox_to_anchor=bbox_loc)
    plt.tight_layout()

    if save == True:
        plt.savefig(savename + ".png", bbox_inches="tight", dpi=600)

    plt.show()


In [ ]:
col_use_list= page_data_pred_inv.columns[-4:]
col_use_list

marker_list= ["o", "v", "s", "X"]

# plt.rcParams["font.family"] = "Arial Unicode MS"
plt.rcParams["font.family"] = "DejaVu Sans"

y_label= "USD2022/tCO" + "\u2082"
y_label

In [ ]:
svr_prtp_folder

In [ ]:
for col_use in col_use_list:
    plot_compare_final(col_use= col_use, 
                       pred_inv_data= page_data_pred_inv, 
                       actual_y_data= PAGE_y_df, 
                       title= "PAGE - " + col_use, 
                       y_label= y_label, 
                       ymin=None, ymax=None, 
                       legend_loc= "upper left",
                       save=True, 
                       savename= svr_prtp_folder + "/" + "pred_actual_PAGE_" + prtp_name + "_" + col_use)

In [ ]:
for col_use in col_use_list:
    plot_compare_final(col_use= col_use, 
                       pred_inv_data= fund_data_pred_inv, 
                       actual_y_data= FUND_y_df, 
                       title= "FUND - " + col_use, 
                       y_label= y_label, 
                       ymin=None, ymax=None, 
                       legend_loc= "upper left",
                       save=True, 
                       savename= svr_prtp_folder + "/" + "pred_actual_FUND_" + prtp_name + "_" + col_use)

In [ ]:
for col_use in col_use_list:
    plot_compare_final(col_use= col_use, 
                       pred_inv_data= dice_data_pred_inv, 
                       actual_y_data= DICE_y_df, 
                       title= "DICE - " + col_use, 
                       y_label= y_label, 
                       ymin=None, ymax=None, 
                       legend_loc= "upper left",
                       save=True, 
                       savename= svr_prtp_folder + "/" + "pred_actual_DICE_" + prtp_name + "_" + col_use)

In [ ]:
plot_compare_final_2(col_use_list= col_use_list, 
                    pred_inv_data= page_data_pred_inv, 
                    actual_y_data= PAGE_y_df, 
                    title= "PAGE - " + prtp_name, 
                    marker_list= marker_list,
                    y_label= y_label, 
                    bbox_loc= (0.2, -0.1),
                    ncol=2,
                    L= 10, H= 8,
                    ymin=None, ymax=None, 
                    legend_loc= "upper left",
                    save=True, 
                    savename=svr_prtp_folder + "/" + "pred_actual_PAGE_" + prtp_name + "_all")

In [ ]:
plot_compare_final_2(col_use_list= col_use_list, 
                    pred_inv_data= fund_data_pred_inv, 
                    actual_y_data= FUND_y_df, 
                    title= "FUND - " + prtp_name, 
                    marker_list= marker_list,
                    y_label= y_label, 
                    bbox_loc= (0.2, -0.1),
                    ncol=2,
                    L= 10, H= 8,
                    ymin=None, ymax=None, 
                    legend_loc= "upper left",
                    save=True, 
                    savename=svr_prtp_folder + "/" + "pred_actual_FUND_" + prtp_name + "_all")

In [ ]:
plot_compare_final_2(col_use_list= col_use_list, 
                    pred_inv_data= dice_data_pred_inv, 
                    actual_y_data= DICE_y_df, 
                    title= "DICE - " + prtp_name, 
                    marker_list= marker_list,
                    y_label= y_label, 
                    bbox_loc= (0.2, -0.1),
                    ncol=2,
                    L= 10, H= 8,
                    ymin=None, ymax=None, 
                    legend_loc= "upper left",
                    save=True, 
                    savename=svr_prtp_folder + "/" + "pred_actual_DCIE_" + prtp_name + "_all")